<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=12e7284912b8d02a0aca44329f0a31aac0b46424b7246e1400720e566b15398b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-07 13:17:45
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.42 C
-------------------
Today PnL: -47.54 K (-0.33%)
Current PnL: -22.10 L (-14.48%)
CY Booked + Current PnL: -9.45 L (-6.19%)
-------------------
Total profit:  1.92 L
Total loss:  -24.02 L
-------------------
Total Booked + Current PnL: 17.53 L (13.96%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.88%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 89.08 L (62.53%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.66,62.0,X-LC,6.43,231145.0,20779.0,7998.0,-0.96,9.88,3.46,13.68,37.0,2.60,1.63,28.03,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.80,173267.0,32427.0,12059.0,1.49,23.02,6.96,31.59,66.0,2.69,1.23,55.61,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-13.84,57.0,X-MC,5.83,148348.0,12215.0,18365.0,0.19,8.97,12.38,22.47,91.0,0.67,1.05,23.12,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.66,36.0,X-MC,2.86,87180.0,-3498.0,18404.0,-1.34,-3.86,21.11,16.44,101.0,-0.19,0.62,15.54,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.0,18.37,62.0,X-SC,6.01,209346.0,18340.0,39357.0,2.67,9.60,18.80,30.21,157.0,0.47,1.48,32.49,X40N,NTT,FINANCE
61,SAIL,228.0,54.77,66.0,M-MC,10.44,243663.0,18701.0,148951.0,2.60,8.31,61.13,74.53,192.0,0.13,1.72,43.25,XY24,BTT,STEEL
84,VOLTAS,1530.0,-5.69,36.0,X-MC,4.39,197895.0,6153.0,31604.0,2.08,3.21,15.97,19.69,99.0,0.19,1.40,10.22,XY25,NTT,AC
34,ICICIPRULI,790.0,-19.50,60.0,X-MC,1.22,138594.0,2806.0,39943.0,1.57,2.07,28.82,31.48,107.0,0.07,0.98,14.44,X40,ATH,INSURANCE
82,VALIANTORG,838.0,-285.28,43.0,H-SC,11.27,102918.0,-60687.0,182000.0,1.56,-37.09,176.84,74.15,139.0,-0.33,0.73,31.92,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SYMPHONY,1306.0,-45.13,41.0,M-SC,16.74,117107.0,-54034.0,53975.0,-4.77,-31.57,46.09,-0.03,196.0,-1.00,0.83,2.10,OX40N,NTT,DURABLES
1,ABB,7934.0,-42.30,36.0,H-LC,5.08,241128.0,-20491.0,139710.0,-3.91,-7.83,57.94,45.57,7.0,-0.15,1.71,2.60,AR,NTT,ELECTRICAL
70,SURYODAY,240.0,60.09,45.0,H-SC,15.83,146299.0,-32772.0,98503.0,-3.83,-18.30,67.33,36.71,135.0,-0.33,1.03,45.16,XR,NTT,BANKS
17,CERA,9475.0,-24.58,41.0,H-SC,1.86,137574.0,-38329.0,80357.0,-3.28,-21.79,58.41,23.89,149.0,-0.48,0.97,18.81,OX40N,NTT,CERAMICS
85,WHIRLPOOL,2270.0,-41.67,48.0,M-SC,6.13,120952.0,6154.0,87884.0,-2.63,5.36,72.66,81.92,223.0,0.07,0.86,43.19,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.92,125584.0,287.0,58635.0,-2.43,0.23,46.69,47.03,141.0,0.00,0.89,57.24,MH,ATH,POWER
56,RECLTD,446.00,42.86,45.0,M-LC,4.72,201520.0,-1650.0,43770.0,1.02,-0.81,21.72,20.74,55.0,-0.04,1.43,5.99,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206081.0,1005.0,111572.0,1.40,0.49,54.14,54.89,52.0,0.01,1.46,24.70,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,103.20,53.0,M-SC,1.94,87521.0,-13256.0,13347.0,0.72,-13.15,15.25,0.09,245.0,-0.99,0.62,14.98,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.75,221310.0,-48357.0,84695.0,-0.92,-17.93,38.27,13.47,138.0,-0.57,1.57,12.53,XY24,NTT,PAINTS
17,CERA,9475.0,-24.58,41.0,H-SC,1.86,137574.0,-38329.0,80357.0,-3.28,-21.79,58.41,23.89,149.0,-0.48,0.97,18.81,OX40N,NTT,CERAMICS
42,JCHAC,2282.0,19921.74,62.0,M-SC,1.51,97877.0,-29928.0,29911.0,-0.01,-23.42,30.56,-0.01,232.0,-1.00,0.69,10.83,OX40N,NTT,AC
67,SIS,528.0,1991.71,42.0,H-SC,2.53,84418.0,-26614.0,50220.0,-2.46,-23.97,59.49,21.26,156.0,-0.53,0.60,14.06,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-3.88,67.0,M-LC,11.8,173267.0,32427.0,12059.0,1.49,23.02,6.96,31.59,66.0,2.69,1.23,55.61,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,54.77,66.0,M-MC,10.44,243663.0,18701.0,148951.0,2.60,8.31,61.13,74.53,192.0,0.13,1.72,43.25,XY24,BTT,STEEL
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206081.0,1005.0,111572.0,1.40,0.49,54.14,54.89,52.0,0.01,1.46,24.70,X5K,ATH,METALS
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.92,125584.0,287.0,58635.0,-2.43,0.23,46.69,47.03,141.0,0.00,0.89,57.24,MH,ATH,POWER
79,UNIONBANK,163.00,-3.88,67.0,M-LC,11.80,173267.0,32427.0,12059.0,1.49,23.02,6.96,31.59,66.0,2.69,1.23,55.61,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,120952.0,6154.0,87884.0,-2.63,5.36,72.66,81.92,223.0,0.07,0.86,43.19,XR,NTT,DURABLES


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,120952.0,6154.0,87884.0,-2.63,5.36,72.66,81.92,223.0,0.07,0.86,43.19,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.71,51.0,H-SC,5.40,129267.0,5931.0,125699.0,-1.38,4.81,97.24,106.72,119.0,0.05,0.91,29.31,AR,ATH,MISC
31,HINDZINC,730.22,26.60,47.0,M-LC,9.80,206081.0,1005.0,111572.0,1.40,0.49,54.14,54.89,52.0,0.01,1.46,24.70,X5K,ATH,METALS
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.92,125584.0,287.0,58635.0,-2.43,0.23,46.69,47.03,141.0,0.00,0.89,57.24,MH,ATH,POWER
86,WIPRO,420.00,-16.52,39.0,M-LC,5.42,146890.0,-4055.0,113517.0,-1.30,-2.69,77.28,72.51,53.0,-0.04,1.04,3.17,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,32.34,148050.0,-128473.0,238553.0,0.33,-46.46,161.13,39.81,54.0,-0.54,1.05,4.35,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR
23,DMART,5391.45,-21.31,25.0,X-LC,7.54,200600.0,-9807.0,68966.0,-1.72,-4.66,34.38,28.12,38.0,-0.14,1.42,17.86,X40N,ATH,FMCG
44,JSWINFRA,342.00,-24.37,26.0,X-MC,4.54,189776.0,-10571.0,41068.0,-0.97,-5.28,21.64,15.23,178.0,-0.26,1.34,25.27,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-14.10,32.0,X-LC,6.41,241200.0,-15863.0,50990.0,-0.21,-6.17,21.14,13.67,24.0,-0.31,1.71,13.87,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.60,35.0,X-MC,0.19,198045.0,-2775.0,70563.0,0.31,-1.38,35.63,33.76,80.0,-0.04,1.40,3.74,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197727.0,-2411.0,23292.0,-0.77,-1.20,11.78,10.44,4.0,-0.10,1.40,4.29,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.61,37.0,H-LC,1.40,151740.0,-34355.0,81833.0,-1.58,-18.46,53.93,25.51,15.0,-0.42,1.07,11.82,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.30,39.0,X-MC,3.19,306075.0,-21916.0,128460.0,1.05,-6.68,41.97,32.48,92.0,-0.17,2.16,3.98,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-6.24,64.0,X-LC,3.96,245702.0,11558.0,37076.0,0.85,4.94,15.09,20.77,86.0,0.31,1.74,12.20,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.42,79320.0,-4190.0,24216.0,-0.25,-5.02,30.53,23.98,82.0,-0.17,0.56,0.51,X40,ATH,APPARELS
53,QUESS,986.00,-50.41,35.0,X-SC,37.41,49308.0,-15698.0,166627.0,0.91,-24.15,337.93,232.18,198.0,-0.09,0.35,0.91,XY24,NTT,MISC
20,COLPAL,3726.84,-7.18,39.0,X-MC,7.29,216500.0,-46865.0,156183.0,-0.40,-17.79,72.14,41.51,84.0,-0.30,1.53,1.60,XY25,ATH,FMCG
1,ABB,7934.00,-42.30,36.0,H-LC,5.08,241128.0,-20491.0,139710.0,-3.91,-7.83,57.94,45.57,7.0,-0.15,1.71,2.60,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,35.0,X-SC,37.41,49308.0,-15698.0,166627.0,0.91,-24.15,337.93,232.18,198.0,-0.09,0.35,0.91,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.85,32.0,X-MC,7.42,79320.0,-4190.0,24216.0,-0.25,-5.02,30.53,23.98,82.0,-0.17,0.56,0.51,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.46,42.0,X-SC,4.21,81216.0,-63944.0,139870.0,-1.23,-44.05,172.22,52.31,136.0,-0.46,0.57,8.71,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.66,36.0,X-MC,2.86,87180.0,-3498.0,18404.0,-1.34,-3.86,21.11,16.44,101.0,-0.19,0.62,15.54,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.58,25.0,X-SC,7.77,87308.0,-41362.0,86662.0,0.11,-32.15,99.26,35.20,219.0,-0.48,0.62,0.21,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.65,44.0,X-LC,9.73,279772.0,-66184.0,132892.0,-1.15,-19.13,47.50,19.28,1.0,-0.50,1.98,3.43,X40,ATH,IT
39,INFY,2275.00,-19.60,49.0,X-LC,4.98,313067.0,179.0,171498.0,0.21,0.06,54.78,54.87,3.0,0.00,2.21,8.34,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197727.0,-2411.0,23292.0,-0.77,-1.20,11.78,10.44,4.0,-0.10,1.40,4.29,X40,NTT,FMCG
83,VBL,671.64,-16.28,54.0,X-LC,6.31,301264.0,-14578.0,127254.0,0.07,-4.62,42.24,35.67,5.0,-0.11,2.13,8.60,X40N,ATH,FMCG
1,ABB,7934.00,-42.30,36.0,H-LC,5.08,241128.0,-20491.0,139710.0,-3.91,-7.83,57.94,45.57,7.0,-0.15,1.71,2.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-49.70,32.0,L-SC,27.47,73344.0,-40205.0,80253.0,-2.58,-35.41,109.42,35.27,268.0,-0.50,0.52,79.51,XR,NTT,HOTELS
7,ASIANTILES,137.00,7355.56,63.0,L-SC,12.30,81781.0,-12029.0,88651.0,-1.97,-12.82,108.40,81.67,269.0,-0.14,0.58,58.37,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,8.34,91770.0,-6210.0,27815.0,0.56,-6.34,30.31,22.05,152.0,-0.22,0.65,33.39,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-41.67,48.0,M-SC,6.13,120952.0,6154.0,87884.0,-2.63,5.36,72.66,81.92,223.0,0.07,0.86,43.19,XR,NTT,DURABLES
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.92,125584.0,287.0,58635.0,-2.43,0.23,46.69,47.03,141.0,0.00,0.89,57.24,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,74.10,64.0,H-SC,4.01,150785.0,-31990.0,143065.0,0.53,-17.50,94.88,60.77,125.0,-0.22,1.07,35.79,XR,NTT,JEWELLERY
68,SONACOMS,806.63,-31.40,68.0,M-SC,6.44,85365.0,-15794.0,55795.0,1.01,-15.61,65.36,39.54,202.0,-0.28,0.60,20.37,AR,ATH,AUTO
46,KPIGREEN,731.05,5.57,50.0,H-SC,7.92,125584.0,287.0,58635.0,-2.43,0.23,46.69,47.03,141.0,0.00,0.89,57.24,MH,ATH,POWER
62,SAMMAANCAP,326.00,-197.73,70.0,M-SC,34.03,168210.0,18990.0,125182.0,0.86,12.73,74.42,96.62,208.0,0.15,1.19,82.04,XY25,NTT,FINANCE
7,ASIANTILES,137.00,7355.56,63.0,L-SC,12.30,81781.0,-12029.0,88651.0,-1.97,-12.82,108.40,81.67,269.0,-0.14,0.58,58.37,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.73
1,25,44.40
2,50,75.58


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.76
LC    33.17
MC    23.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.28
X40N     12.47
XY25     11.96
XR        9.88
AR        7.78
OX40N     6.50
X200      1.77
X5K       1.46
SR        1.04
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.22
X-LC    22.48
X-MC    18.76
M-SC    12.57
X-SC     5.54
M-LC     5.16
H-LC     4.55
H-MC     2.32
M-MC     1.72
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.29,-4.76,38.63
IT,12.43,-19.67,83.78
FINANCE,12.19,-9.74,59.48
MISC,7.78,-18.03,77.57
ELECTRICAL,5.71,-11.94,51.63
PAINTS,5.60,-16.94,34.37
BANKS,4.04,-18.35,87.19
INSURANCE,3.90,-0.36,37.77
AUTO,2.73,-49.08,111.30


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3333025.0,22
XR,1261703.0,13
AR,1185581.0,9
X40,855636.0,12
X40N,705182.0,9
OX40N,564946.0,9
XY25,483287.0,8
SR,274217.0,2
X5K,111572.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407440.0,24
M-SC,1267246.0,15
X-MC,1222568.0,14
X-LC,1102980.0,13
X-SC,565969.0,6
H-LC,295967.0,3
M-LC,280918.0,4
L-SC,259137.0,3
H-MC,259061.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197946.0      6
           AR         840706.0      5
           XR         784839.0      7
M-SC       XY24       734145.0      6
X-MC       XY24       575624.0      4
X-LC       X40        469023.0      5
           XY24       314526.0      2
X-SC       X40N       312680.0      4
X-MC       X40        299951.0      6
H-SC       SR         274217.0      2
           OX40N      251097.0      3
X-LC       X40N       233296.0      3
H-LC       AR         221543.0      2
X-MC       XY25       187787.0      2
H-MC       XY24       183147.0      1
L-SC       XR         168904.0      2
X-SC       XY24       166627.0      1
X-MC       X40N       159206.0      2
M-MC       XY24       148951.0      1
M-SC       OX40N      147702.0      4
           XR         136885.0      2
           XY25       125182.0      1
           AR         123332.0      2
M-LC       XR         113517.0      1
           X5K        111572.0      1
L-SC       OX40N       90233.0      1
X-SC       X40         86662.0      1
X-LC       XY25        86135.0      3
H-MC       OX40N       75914.0      1
H-LC       X200        74424.0      1
H-SC       MH          58635.0      1
L-MC       XR          57558.0      1
M-LC       XY25        43770.0      1
L-LC       XY25        40413.0      1
M-LC       XY24        12059.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
